In [ ]:
import pandas as pd
from agent.qdrant import fill
from agent.config import config
import nest_asyncio
nest_asyncio.apply()

all_lang = {
    "ja": 1,
    # 2 is incomplete
    "ko": 3,
    "zh-Hant": 4,
    "fr": 5,
    "de": 6,
    "es": 7,
    "it": 8,
    "en": 9,
    # 10 is missing
    # 11 is a duplicate
    "zh-Hans": 12
}

all_names = pd.read_csv("pokemon_species_names.csv")
all_flavors = pd.read_csv("pokemon_species_flavor_text.csv")

for lang in config["languages"]["supported"]:
    names = all_names[all_names["local_language_id"] == all_lang[lang]]
    flavors = all_flavors[all_flavors["language_id"] == all_lang[lang]]
    await fill(
        names,
        flavors,
        lang,
    )